# Refactored HistoryMatching workflow example

## 1. Set up

In [1]:
import torch

# imports from main
from autoemulate.history_matching_dashboard import HistoryMatchingDashboard

# imports from experimental
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)
from autoemulate.experimental.simulations.epidemic import Epidemic
from autoemulate.experimental.history_matching import HistoryMatching, HistoryMatchingWorkflow

### Simulate data & train a GP

Set up a Simulator and generate data.

In [2]:
simulator = Epidemic()
x = simulator.sample_inputs(10)
y = simulator.forward_batch(x)

Running simulations: 100%|██████████| 10/10 [00:00<00:00, 859.54it/s]

Successfully completed 10/10 simulations (100.0%)


The next step should be done with `AutoEmulate.compare()`.

In [3]:
gp_pytorch = GaussianProcessExact(x, y)
gp_pytorch.fit(x, y)

### Generate predictions

In [4]:
x = simulator.sample_inputs(10)
output = gp_pytorch.predict(torch.tensor(x, dtype=torch.float32))
pred_means, pred_vars = (
    output.mean.float().detach(),
    output.variance.float().detach(),
)

/var/folders/bj/kdwy1bhj3h728lr5xdj19yd40000gr/T/ipykernel_3664/2686381175.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = gp_pytorch.predict(torch.tensor(x, dtype=torch.float32))


## 2. HistoryMatching

Firstly, one can instantiate HistoryMatching without a simulator or an emulator. It can be used to calculate implausability for a given set of predictions.

In [5]:
# Define observed data with means and variances
observations = {"infection_rate": (0.3, 0.05)}

# Create history matcher
hm = HistoryMatching(
    observations=observations,
    threshold=3.0
)

implausability = hm.calculate_implausibility(pred_means, pred_vars)


Once implausability has been calculated, it can be used to identify indices of NROY parameters:

In [6]:
hm.get_nroy(implausability)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Or to filter out parameters at those NROY indices:

In [7]:

hm.get_nroy(implausability, x)

tensor([[0.2698, 0.1451],
        [0.2420, 0.1100],
        [0.3180, 0.1714],
        [0.1243, 0.0569],
        [0.4924, 0.1418],
        [0.3767, 0.1907],
        [0.2018, 0.0157],
        [0.1417, 0.0442],
        [0.4258, 0.0866],
        [0.3977, 0.0744]])

## 3. Iterative HistoryMatchingWorkflow

We also have a separate class that implements an iterative sample-predict-evaluate workflow:
- sample `n_test_samples` to test from the NROY space
- use emulator to filter out implausible samples and update the NROY space
- make `n_simulation_samples` predictions for the sampled parameters using the simulator
- refit the emulator using the simulated data

The object maintains and updates the internal state each time `run()` is called so this can be done as many times as the user wants.

In [8]:
hmw = HistoryMatchingWorkflow(
    simulator=simulator,
    emulator=gp_pytorch,
    observations=observations,
    threshold=3.0,
    train_x=x,
    train_y=y
)

parameters, impl_scores = hmw.run(n_simulation_samples=20, n_test_samples=100)

Running simulations: 100%|██████████| 20/20 [00:00<00:00, 1138.30it/s]

Successfully completed 20/20 simulations (100.0%)


## 4. Integration with dashboard

In [9]:
dashboard = HistoryMatchingDashboard(
    samples=parameters,
    impl_scores=impl_scores,
    param_names=simulator.param_names,  
    output_names=simulator.output_names, 
    )

In [10]:
dashboard.display()

HTML(value='<h2>History Matching Dashboard</h2>')